In [2]:
# import
import importlib
import pandas as pd
from pipeline_executor import PipelineExecutor
import nlp_analysis.word_wizard as ww

topic = "'quantumcomputing'AND'research'"

pipeline_executor = PipelineExecutor()
quantum_df = pipeline_executor.execute(query=topic, max_articles=20, overwrite=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 5.7 MB/s eta 0:00:0000:0100:01m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached umap-learn-0.5.3.tar.gz (88 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 5.6 MB/s eta 0:00:0000:0100:01
  Using cached Cython-0.29.35-py2.py3-none-any.whl (988 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 3.6 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.0 MB/s eta 0:00:0000:0100:01m
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.4/30.4 MB 3.8 MB/s eta 0:00:0000:0100:01
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp311-cp311-macosx_10_9_universal2.whl size=1283943 sha256=83a75341df3592069812ad5ca31356c55d8c

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Library/Frameworks/Python.framework/V

In [ ]:
# safe and load quantum_df using parquet
quantum_df.to_parquet('quantum_df.parquet.gzip', compression='gzip')
quantum_df = pd.read_parquet('quantum_df.parquet.gzip')

In [114]:
importlib.reload(ww)
wizard = ww.WordWizard(df=quantum_df, interest = 'body')
wizard.create_word_embeddings(wizard.interest, lean=True)
wizard.cluster_embeddings(wizard.interest, k=3)
wizard.entitiy_recognition(wizard.interest)
# wizard.summarize_medoids(wizard.interest)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Creating word embeddings for body: 100%|██████████| 30/30 [00:57<00:00,  1.91s/it]


In [115]:
wizard.df

,engine,link,source,title,description,body,body_word_embeddings,body_word_embeddings_clusters,body_word_embeddings_clusters_medoids,body_clusters_word_embeddings_NER
0,Bing,https://www.slu.edu/nursing/research-centers/i...,Saint Louis University,Nursing Research and Scholarship,The Trudy Busch Valentine School of Nursing se...,The Trudy Busch Valentine School of Nursing se...,"[0.2794007, -0.31897348, -0.025907502, 0.19047...",2,True,None
1,Bing,https://www.slu.edu/research/about-slu-researc...,Saint Louis University,Research Councils and Committees,The Research Growth Committee internally advis...,The Research Growth Committee The Research Gr...,"[0.12197025, -0.32317457, 0.12367886, 0.153825...",2,False,None
2,Bing,https://hackaday.com/2015/09/29/quantum-comput...,Hackaday,Quantum Computing Kills Encryption,Imagine a world where the most widely-used cry...,Imagine a world where the most widely-used cry...,"[0.3591106, -0.0269114, 0.013930714, -0.082127...",0,False,None
3,Bing,https://www.healthline.com/nutrition/melatonin...,Healthline,"Melatonin for Sleep: Uses, Dosage, Side Effect...",There is strong evidence for the benefits mela...,There is strong evidence for the benefits mela...,"[0.30619758, -0.18140519, 0.092418805, 0.23314...",0,False,None
4,Bing,https://www.bristol.ac.uk/red/,University of Bristol,Research and Enterprise Division,Research and Enterprise Division (RED) works w...,Research and Enterprise Division (RED) Resear...,"[0.3591871, 0.041275326, -0.16974252, 0.151446...",1,False,None
5,Bing,https://www.morningstar.com/research/signature,Morningstar,Investment Research Methodology,The grades in the Global Investor Experience F...,Latest Investment Research to Download Method...,"[0.29742873, -0.26362938, -0.13502364, 0.05558...",2,False,None
6,Bing,https://phys.org/news/2023-06-atomsa-block-qua...,Phys.org,The 'breath' between atoms—a new building bloc...,The researchers also developed a device that c...,This article has been reviewed according to Sc...,"[0.50547737, -0.13992067, 0.047450535, -0.1080...",0,False,None
7,Bing,https://www.politico.eu/article/germany-unveil...,Politico Europe,Germany unveils powerful quantum computer to k...,Angela Merkel on Tuesday inaugurated a compute...,Angela Merkel on Tuesday inaugurated a compute...,"[0.23366344, -0.107322395, -0.061297044, 0.007...",1,True,None
8,Bing,https://www.slu.edu/medicine/pathology/microsc...,Saint Louis University,Research Microscopy and Histology Core,The Research Microscopy and Histology Core pro...,The Research Microscopy and Histology Core pro...,"[0.47896588, -0.31212205, 0.07406148, 0.025443...",2,True,None
9,Bing,https://www.cnbc.com/2023/04/20/alphabet-merge...,CNBC,Read the internal memo Alphabet sent in mergin...,"Alphabet is merging Google Brain, part of the ...","Jeff Dean, head of artificial intelligence at ...","[0.34225625, -0.13923632, -0.09666089, 0.01155...",1,False,None


In [113]:
self = wizard
column = 'body'
self.df[self.df[column + self.EMB_SUFFIX + self.CLUSTER_SUFFIX] == 1][column + self.CLUSTER_SUFFIX + self.EMB_SUFFIX + self.NER_SUFFIX] = 6

In [93]:
# check the number of entities in each cluster
wizard.df['body_word_embeddings_clusters'].value_counts()

1    13
2    11
0     6
Name: body_word_embeddings_clusters, dtype: int64

In [94]:
# check the number of medoids per cluster
wizard.df[wizard.df['body_word_embeddings_clusters_medoids'] == True][['body_word_embeddings', 'body_word_embeddings_clusters']]

,body_word_embeddings,body_word_embeddings_clusters
0,"[0.2794007, -0.31897348, -0.025907502, 0.19047...",0
7,"[0.23366344, -0.107322395, -0.061297044, 0.007...",1
8,"[0.47896588, -0.31212205, 0.07406148, 0.025443...",0
10,"[0.35769367, -0.2094464, -0.10805304, -0.03745...",1
23,"[0.41990936, -0.17660353, 0.05122102, -0.02868...",2
25,"[0.34437597, -0.14016607, 0.1357093, -0.023257...",2


In [96]:
see = wizard.df[wizard.df['body_word_embeddings_clusters_medoids'] == True][['body_word_embeddings_clusters', 'body_word_embeddings_clusters_medoids_summaries']]